In [ ]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import requests
import pymongo
import datetime
import json
import requests
import time
from bson import ObjectId

In [10]:
url = 'https://www.worldometers.info/coronavirus/'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
soup = BeautifulSoup(r.text,'lxml')

In [11]:
trs = soup.find_all('tr')

In [12]:
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.country_coords
coordsList = collection.find_one()['countries']

In [13]:
# Loop through countries
countries = []

names = []
totalCasesList = []
deathsList = []
recoveredList = []
activeCasesList = []
testsList = []
populationList = []
coordsData = []
newCasesList = []

for i in range(len(trs)):
    # If country row, pull data
    if 'href="country' in str(trs[i]):
        country = str(trs[i]).split('\n')
        name = country[2].split('</a>')[0].split("/\">")[1]
        totalCases = country[3].split('>')[1].split('<')[0]
        deaths = country[5].split("\">")[1].split('<')[0]
        recovered = country[8].split("\">")[1].split('<')[0]
        activeCases = country[10].split("\">")[1].split('<')[0]
        tests = country[14].split("\">")[1].split('<')[0]
        population = country[16].split('>')[2].split('<')[0]
        
        #Prevent indexing problems if no coords found
        lat = 0
        lng = 0
        
        for x in range(len(coordsList)):
            if coordsList[x]['name'] == name:
#                 print(coordsList[x])
                lat = coordsList[x]['lat']
                lng = coordsList[x]['lng']
                break
        
        
        try:
            newCases = country[4].split('>+')[1].split('</')[0]
        except:
            newCases = 0
#         print(name,totalCases,deaths,recovered,activeCases,tests,population,newCases)
        
        names.append(name)
        totalCasesList.append(totalCases)
        deathsList.append(deaths)
        activeCasesList.append(activeCases)
        recoveredList.append(recovered)
        populationList.append(population)
        testsList.append(tests)
        coordsData.append([lat,lng])
        newCasesList.append(newCases)
        
        
#         print('\n')
        i+=1


In [14]:
countryDict = {
    'name': names,
    'totalCases': totalCasesList,
    'deaths': deathsList,
    'recovered': recoveredList, 
    'activeCases': activeCasesList,
    'population': populationList,
    'coords': coordsData,
    'newCases': newCasesList
}
countryDict

{'name': ['USA',
  'Brazil',
  'India',
  'Russia',
  'South Africa',
  'Peru',
  'Mexico',
  'Chile',
  'Spain',
  'UK',
  'Iran',
  'Pakistan',
  'Saudi Arabia',
  'Italy',
  'Turkey',
  'Bangladesh',
  'Colombia',
  'Germany',
  'France',
  'Argentina',
  'Canada',
  'Qatar',
  'Iraq',
  'Egypt',
  'Indonesia',
  'Sweden',
  'Ecuador',
  'Kazakhstan',
  'Philippines',
  'Oman',
  'Belarus',
  'Belgium',
  'Bolivia',
  'Kuwait',
  'Ukraine',
  'UAE',
  'Panama',
  'Dominican Republic',
  'Israel',
  'Netherlands',
  'Portugal',
  'Singapore',
  'Poland',
  'Guatemala',
  'Romania',
  'Nigeria',
  'Bahrain',
  'Afghanistan',
  'Armenia',
  'Honduras',
  'Switzerland',
  'Ghana',
  'Azerbaijan',
  'Kyrgyzstan',
  'Ireland',
  'Japan',
  'Algeria',
  'Serbia',
  'Moldova',
  'Austria',
  'Nepal',
  'Morocco',
  'Uzbekistan',
  'Cameroon',
  'Ivory Coast',
  'Czechia',
  'S. Korea',
  'Kenya',
  'Denmark',
  'Australia',
  'Venezuela',
  'El Salvador',
  'Costa Rica',
  'Sudan',
  'Ethio

In [15]:
# Update data in db
collection = db.corona_data

In [16]:
updatedData = [countryDict]

In [17]:
# res = collection.find({"_id" : ObjectId("5f1201553772928119aabb9f")})
# for r in res:
#     print(r)

In [18]:
collection.update_one({},{"$set":{"countries":updatedData}})

In [ ]:
# with open('sample.json', 'w') as f:
#     json.dump(updatedData, f)
